In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

def spearsonr_pval(x,y):
    return spearmanr(x,y)[1]

data_full = pd.read_csv("RVT-142DPI-metadata-feature-table.csv")
data_a = data_full[data_full["Treatmentgroup"].isin(["Group1C","Group3E"])]

data_not_restore = pd.read_csv("RVT-mass-not-restored.csv")
data_not_restore_list = data_not_restore.filename.to_list()
data_not_restore_list.insert(0, "MouseID")

# this is metadata info, which is extra baylor information.
# careful for the range, [a,b] mean a <= X < b. Also python is 0 based index.
data_M = data_a.iloc[:, np.r_[0, 2:9]]
data_M = data_M.set_index('MouseID')
# this is metabolites, which is features.
data_F = data_a[data_not_restore_list]
data_F = data_F.set_index('MouseID')

# get the two part of column names: metadata colum name and features column name.
M_column_list = data_M.head(0)
print(M_column_list)
F_column_list = data_F.head(0)
print(F_column_list)

df_col_list = []
df_corr_list = []
df_pval_list = []

for m_col in M_column_list:
    for f_col in F_column_list:
        col_1 = data_M[m_col]
        col_2 = data_F[f_col]
        corr_col_1_2 = col_1.corr(col_2, method='spearman')
        corr_pval_col_1_2 = col_1.corr(col_2, method=spearsonr_pval)
        df_col_list.append(m_col + "-" + f_col)
        df_corr_list.append(corr_col_1_2)
        df_pval_list.append(corr_pval_col_1_2)

print(df_col_list)
print(df_corr_list)
print(df_pval_list)

import statsmodels.api

df_final = pd.DataFrame(list(zip(df_col_list, df_corr_list, df_pval_list)), columns=['col_name','corr', 'corr_pval'])

df_final['corr'].replace('', float("NaN"), inplace=True)
df_final.dropna(subset=['corr'], inplace=True)

dr_fdr_pval = statsmodels.stats.multitest.fdrcorrection(list(df_final["corr_pval"]), method='indep')
df_final['FDR_rejected'] = dr_fdr_pval[0]
df_final['FDR_pval'] = dr_fdr_pval[1]

df_final.to_csv('RVT-142DPI-not-restored-corr-pval.csv', index=False)

Empty DataFrame
Columns: [Heart.Weight..Body.Weight, Liver.Weight..Body.Weight, Ejection.Fraction, PR.Interval..s., QTc..s., P.Amplitude..V., CD3.CD8.IFNg.]
Index: []
Empty DataFrame
Columns: [X137.046_0.35, X172.134_2.343, X214.181_2.634, X228.196_2.71, X230.176_2.586, X242.212_2.78, X256.227_2.847, X256.227_3.318, X258.207_2.775, X268.105_0.35, X269.088_0.35, X272.222_2.957, X278.248_3.1, X280.264_3.159, X282.28_3.246, X296.259_2.971, X306.279_2.989, X306.279_3.212, X308.295_3.045, X308.295_3.285, X352.321_3.187, X368.316_2.962, X399.394_3.159, X399.395_3.152, X530.289_2.929, X666.434_3.024, X705.141_2.339, X776.253_2.398, X776.411_2.398, X811.309_2.773, X811.31_2.767, X819.322_2.398]
Index: []

[0 rows x 32 columns]
['Heart.Weight..Body.Weight-X137.046_0.35', 'Heart.Weight..Body.Weight-X172.134_2.343', 'Heart.Weight..Body.Weight-X214.181_2.634', 'Heart.Weight..Body.Weight-X228.196_2.71', 'Heart.Weight..Body.Weight-X230.176_2.586', 'Heart.Weight..Body.Weight-X242.212_2.78', 'Heart.We